# RL basics

Термины и понятия:

- агент/среда
- наблюдение $o$ / состояние $s$
- действие $a$, стратегия $\pi: \pi(s) \rightarrow a$ функция перехода $T: T(s, a) \rightarrow s'$
- вознаграждение $r$, ф-я вознаграждений $R: R(s, a) \rightarrow r$
- цикл взаимодействия, траектория $\tau: (s_0, a_0, r_0, s_1, a_1, r_1, ..., s_T, a_T, r_T)$, эпизод
- отдача $G$, подсчет отдачи, средняя[/ожидаемая] отдача $\mathbb{E}[G]$

In [64]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [65]:
import glob
import io
import base64
import gymnasium as gym
import numpy as np
from IPython import display as ipythondisplay
from IPython.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline

## Agent, environment

<img src=https://gymnasium.farama.org/_images/lunar_lander.gif caption="lunar lander" width="150" height="50"><img src=https://gymnasium.farama.org/_images/mountain_car.gif caption="mountain car" width="150" height="50">
<img src=https://gymnasium.farama.org/_images/cliff_walking.gif caption="cliff walking" width="300" height="50">
<img src=https://ale.farama.org/_images/montezuma_revenge.gif caption="montezuma revenge" width="150" height="100">
<img src=https://github.com/danijar/crafter/raw/main/media/video.gif caption="crafter" width="150" height="100">
<img src=https://camo.githubusercontent.com/6df2ca438d8fe8aa7a132b859315147818c54af608f8609320c3c20e938acf48/68747470733a2f2f6d656469612e67697068792e636f6d2f6d656469612f344e78376759694d394e44724d724d616f372f67697068792e676966 caption="malmo minecraft" width="150" height="100">
<img src=https://images.ctfassets.net/kftzwdyauwt9/e0c0947f-1a44-4528-4a41450a9f0a/2d0e85871d58d02dbe01b2469d693d4a/table-03.gif caption="roboschool" width="150" height="100">
<img src=https://raw.githubusercontent.com/Tviskaron/mipt/master/2019/RL/02/mdp.png caption="Марковский процесс принятия решений" width="150" height="100">
<img src=https://minigrid.farama.org/_images/DoorKeyEnv.gif caption="minigrid" width="120" height="120">

## Observation, state

TODO:
- добавить примеры наблюдений/состояний (числа, векторы, картинки)
- интуитивное объяснение различия, положить пока, что наблюдение = состояние
- пространство состояний


В каждый момент времени среда имеет некоторое внутреннее состояние. Здесь слово "состояние" я употребил скорее в интуитивном понимании, чтобы обозначить, что среда изменчива (иначе какой смысл с ней взаимодействовать, если ничего не меняется). В обучении с подкреплением под термином состояние $s$ (или $s_t$, где $t$ — текущее время) подразумевают либо абстрактно информацию о "состоянии" среды, либо ее явное представление в виде данных, достаточные для полного описания "состояния". *NB: Здесь можно провести аналогию с компьютерными играми — файл сохранения игры как раз содержит информацию о "состоянии" мира игры, чтобы в будущем можно было продолжить с текущей точки, так что данные этого файла в целом можно с некоторой натяжкой считать состоянием (с натяжкой, потому что редко когда в сложных играх файлы сохранения содержат прямо вот всю информацию, так что после перезагрузки вы получите не совсем точную копию). При этом обычно подразумевается, что состояние не содержит в себе ничего лишнего, то есть это **минимальный** набор информации.*

Наблюдением $o$ называют то, что агент "видит" о текущем состоянии среды. Это не обязательно зрение, а вообще вся доступная ему информация (условно, со всех его органов чувств).

В общем случае наблюдение: кортеж/словарь многомерных векторов чисел.

In [66]:
print(gym.make("CartPole-v0").reset()[0].shape)
print(gym.make("MountainCar-v0").reset()[0].shape)

(4,)
(2,)


/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


## Action, policy, transition function

Рассмотрим следующие MDP:

- A: <img src=https://i.ibb.co/mrCMVZLQ/mdp-a.png caption="A" width="400" height="100">
- B: <img src=https://i.ibb.co/GQ2tVtjC/mdp-b.png caption="B" width="400" height="100">

Links to all:
[A](https://i.ibb.co/mrCMVZLQ/mdp-a.png)
[B](https://i.ibb.co/GQ2tVtjC/mdp-b.png)
[C](https://i.ibb.co/Jj9LYHjP/mdp-c.png)
[D](https://i.ibb.co/Y47Mr83b/mdp-d.png)
[E](https://i.ibb.co/Kjt1Xhmf/mdp-e.png)

Давайте явно запишем пространства состояний $S$ и действий $A$, а также функцию перехода $T$ среды.

In [68]:
states_a = set(range(3))
actions_a = set(range(1))
T_a = {
    (0, 0): 1,
    (1, 0): 2,
    (2, 0): 2
}

states_b = set(range(4))
actions_b = set(range(3))
T_b = {
    (0, 0): 1, (0, 1): 2, (0, 2): 0,
    (1, 0): 1, (1, 1): 1, (1, 2): 1,
    (2, 0): 2, (2, 1): 2, (2, 2): 2,
    (3, 0): 3, (3, 1): 3, (3, 2): 3
}

states_c = set(range(4))
actions_c = set(range(2))
T_c = {
    (0, 0): 1, (0, 1): 2,
    (1, 0): 1, (1, 1): 3,
    (2, 0): 3, (2, 1): 2,
    (3, 0): 3, (3, 1): 3
}

states_d = set(range(3))
actions_d = set(range(1))
T_d = {
    (0, 0): [0.2, 0.8, 0.0],
    (1, 0): [0.0, 0.3, 0.7],
    (2, 0): [0.0, 0.0, 1.0],
}

states_e = set(range(4))
actions_e = set(range(2))
T_e = {
    (0, 0): [0.0, 0.8, 0.2, 0.0], (0, 1): [0.0, 0.1, 0.9, 0.0],
    (1, 0): [0.0, 0.2, 0.0, 0.8], (1, 1): [0.0, 0.0, 1.0, 0.0], # or ..., (1, 1): 2
    (2, 0): [0.0, 1.0, 0.0, 0.0], (2, 1): [0.0, 0.0, 0.3, 0.7], # or (2, 0): 1, ...
    (3, 0): [0.0, 0.0, 0.0, 1.0], (3, 1): [0.0, 0.0, 0.0, 1.0], # or (3, 0): 3, (3, 1): 3
}

Попробуйте записать функцию перехода в матричном виде:

In [69]:
P_a = np.array([
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 0.0, 1.0]
])
P_b = np.array([
    [0.0, 1/3, 1/3, 1/3],
    [0.0, 1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0],
    [0.0, 0.0, 0.0, 1.0]
])
P_c = np.array([
    [0.0, 1/2, 1/2, 0.0],
    [0.0, 1/2, 0.0, 1/2],
    [0.0, 0.0, 1/2, 1/2],
    [0.0, 0.0, 0.0, 1.0]
])
P_d = np.array([
    [0.2, 0.8, 0.0],
    [0.0, 0.3, 0.7],
    [0.0, 0.0, 1.0],
])
P_e = np.array([
    [0.0, 0.45, 0.55, 0.0],
    [0.0, 0.1, 0.5, 0.4],
    [0.0, 0.5, 0.15, 0.35],
    [0.0, 0.0, 0.0, 1.0]
])

Как получить вероятность нахождения агента в состоянии (1) через N шагов? Что происходит с вероятностями нахождения в состояниях при $N \rightarrow \infty$

In [70]:
def P_N_steps(p0: np.array, P: np.array, N: int) -> np.array:
    return p0 @ np.linalg.matrix_power(P, N)
# Пусть у нас первоначальное распределение между состояниями - равномерно, тогда
print('A: ', P_N_steps([1/3, 1/3, 1/3], P_a, 10000))
print('B: ', P_N_steps([1/4, 1/4, 1/4, 1/4], P_b, 10000))
print('C: ', P_N_steps([1/4, 1/4, 1/4, 1/4], P_c, 10000))
print('D: ', P_N_steps([1/3, 1/3, 1/3], P_d, 10000))
print('E: ', P_N_steps([1/4, 1/4, 1/4, 1/4], P_e, 10000))

A:  [0. 0. 1.]
B:  [0.         0.33333333 0.33333333 0.33333333]
C:  [0. 0. 0. 1.]
D:  [0. 0. 1.]
E:  [0. 0. 0. 1.]


Итак, во всех средах, кроме B, вероятность оказаться в состоянии 1 стремится к нулю

Давайте попробуем задать двух агентов: случайного и оптимального (для каждой среды свой).

In [92]:
class Agent:
    def __init__(self, actions):
        self.rng = np.random.default_rng()
        self.actions = np.array(list(actions))

    def act(self, state):
        return self.rng.integers(len(self.actions))

class OptimalAgentA:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        return 0

class OptimalAgentB:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        return 1

class OptimalAgentC:
    def __init__(self, actions):
        self.actions = np.array(list(actions))

    def act(self, state):
        if state == 0: return 0
        if state == 1: return 1
        if state == 2: return 0

В качестве дополнения, запишите стратегию агента

## Reward, reward function

Теперь добавим произвольную функцию вознаграждения. Например, для A:

In [72]:
R_a = {(0, 0): -0.1, (1, 0): 1.0, (2, 0): 0.0}

R_b = {
    (0, 0): 0.0, (0, 1): 1.0, (0, 2): 0.0,
    (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0,
    (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0,
    (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0,
}

R_c = {
    (0, 0): -0.1, (0, 1): +0.1,
    (1, 0): -0.5, (1, 1): +1.0,
    (2, 0): -0.1, (2, 1): -0.5,
    (3, 0): 0.0, (3, 1): 0.0,
}

In [74]:
terminal_states_a = {2}
terminal_states_b = {1, 2, 3}
terminal_states_c = {3}

In [75]:
A_mdp = states_a, actions_a, T_a, R_a, terminal_states_a
B_mdp = states_b, actions_b, T_b, R_b, terminal_states_b
C_mdp = states_c, actions_c, T_c, R_c, terminal_states_c

## Interaction loop, trajectory, termination, truncation, episode

Общий цикл взаимодействия в рамках эпизода:
1. Инициализировать среду: $s \leftarrow \text{env.init()}$
2. Цикл:
    - выбрать действие: $a \leftarrow \pi(s)$
    - получить ответ от среды: $s, r, d \leftarrow \text{env.next(a)}$
    - если $d == \text{True}$, выйти из цикла

In [83]:
def run_episode(mdp):
    states, actions, T, R, terminal_states = mdp
    agent = Agent(actions)

    s = 0
    tau = []
    for _ in range(5):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]

        tau.append((s, a, r))
        s = s_next

    return tau

run_episode(A_mdp)

[(0, np.int64(0), -0.1),
 (1, np.int64(0), 1.0),
 (2, np.int64(0), 0.0),
 (2, np.int64(0), 0.0),
 (2, np.int64(0), 0.0)]

Termination — означает окончание эпизода, когда достигнуто терминальное состояние. Является частью задания среды.

Truncation — означает окончание эпизода, когда достигнут лимит по числу шагов (=времени). Обычно является внешне заданным параметром для удобства обучения.

Пока не будем вводить truncation, но поддержим termination: расширьте определение среды информацией о терминальных состояниях для всех описанных ранее сред. Сгенерируйте по несколько случайных траекторий для каждой среды.

In [85]:
def run_episode_with_policy(mdp, strategy="random", truncation=10):
    states, actions, T, R, terminal_states = mdp

    if strategy == "random":
        agent = Agent(actions)
    else:
        if mdp == A_mdp:
            agent = OptimalAgentA(actions)
        elif mdp == B_mdp:
            agent = OptimalAgentB(actions)
        elif mdp == C_mdp:
            agent = OptimalAgentC(actions)

    s = 0
    tau = []
    for step in range(truncation):
        a = agent.act(s)
        s_next = T[(s, a)]
        r = R[(s, a)]
        terminated = (s_next in terminal_states)

        tau.append((s, a, r, s_next, terminated))
        s = s_next
        if terminated: break

    return tau

In [93]:
# Тестируем случайного агента для всех сред
print("Случайные агенты")
for env_name, mdp in [("A", A_mdp), ("B", B_mdp), ("C", C_mdp)]:
    print(f"\nСреда {env_name}:")
    for i in range(5):
        tau = run_episode_with_policy(mdp, strategy="random", truncation=10)
        total_reward = sum(r for _,_,r,_,_ in tau)
        print(f"  Траектория {i+1}: {len(tau)} шагов, награда: {total_reward:.1f}")

print("\nОптимальные агенты")
for env_name, mdp in [("A", A_mdp), ("B", B_mdp), ("C", C_mdp)]:
    tau = run_episode_with_policy(mdp, strategy="optimal", truncation=10)
    total_reward = sum(r for _,_,r,_,_ in tau)
    print(f"Среда {env_name}: {len(tau)} шагов, награда: {total_reward:.1f}")

Случайные агенты

Среда A:
  Траектория 1: 2 шагов, награда: 0.9
  Траектория 2: 2 шагов, награда: 0.9
  Траектория 3: 2 шагов, награда: 0.9
  Траектория 4: 2 шагов, награда: 0.9
  Траектория 5: 2 шагов, награда: 0.9

Среда B:
  Траектория 1: 1 шагов, награда: 0.0
  Траектория 2: 1 шагов, награда: 1.0
  Траектория 3: 2 шагов, награда: 0.0
  Траектория 4: 3 шагов, награда: 0.0
  Траектория 5: 3 шагов, награда: 0.0

Среда C:
  Траектория 1: 4 шагов, награда: -1.0
  Траектория 2: 3 шагов, награда: -0.5
  Траектория 3: 2 шагов, награда: 0.9
  Траектория 4: 2 шагов, награда: 0.0
  Траектория 5: 3 шагов, награда: 0.4

Оптимальные агенты
Среда A: 2 шагов, награда: 0.9
Среда B: 1 шагов, награда: 1.0
Среда C: 2 шагов, награда: 0.9


### Return, expected return

Наиболее важная метрика оценки качества работы агента: отдача.

Отдача: $G(s_t) = \sum_{i=t}^T r_i$

Обычно также вводят параметр $\gamma \in [0, 1]$, дисконтирующий будущие вознаграждения. А еще, тк отдача может меняться от запуска к запуску благодаря вероятностным процессам, нас интересует отдача в среднем — ожидаемая отдача:

$$\hat{G}(s_t) = \mathbb{E} [ \sum_{i=t}^T \gamma^{i-t} r_i ]$$

Именно ее и оптимизируют в RL.

Давайте научимся считать отдачу для состояний по траектории и считать среднюю отдачу.

In [98]:
def compute_return(mdp, n_runs=1000, gamma=0.7):
    rewards_list = []
    for _ in range(n_runs):
        tau = run_episode_with_policy(mdp, "random", truncation=100)
        rewards = [r for _,_,r,_,_ in tau]
        rewards_list.append(rewards)

    max_len = max(len(r) for r in rewards_list)
    total_rewards = np.zeros(max_len)

    for rewards in rewards_list:
        padded = np.pad(rewards, (0, max_len - len(rewards)), 'constant')
        total_rewards += padded

    avg_rewards = total_rewards / n_runs
    discounts = gamma ** np.arange(max_len)

    return np.sum(avg_rewards * discounts)

G_hat_A = compute_return(A_mdp)
G_hat_B = compute_return(B_mdp)
G_hat_C = compute_return(C_mdp)

print(f'{G_hat_A=:.3f}')
print(f'{G_hat_B=:.3f}')
print(f'{G_hat_C=:.3f}')

G_hat_A=0.600
G_hat_B=0.444
G_hat_C=-0.004
